In [1]:
import pandas as pd

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf
from sklearn.metrics import confusion_matrix, roc_auc_score

In [ ]:
# Collect predictions

def check_num_epochs(run):
    num_epochs = 40
    if Path(run / "valid_log.csv").is_file():
        valid_log = pd.read_csv(run / "valid_log.csv", index_col=0)
        if len(valid_log) != num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    
    mode = cfg['data']['validation']['mode']
    seed = cfg['data']['validation'].get('split_seed', cfg.get('seed', -1))
    
    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path, index_col=0)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')
    
    data['model'] = model_name
    data['split_seed'] = seed
    data['mode'] = mode
    
    return data

def collect_all(model_name, root, csv_file):
    root = Path(root)
    
    metrics = [collect_one(model_name, run, csv_file) for run in list(root.glob("run-*"))]
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [ ]:
# Compute metrics for each detected run

def compute_metrics(data, grouping):
    columns = ['Model', 'Mode', 'Split Seed', 'Accuracy', 'Precision', 'Recall', 'F1', 'False Alarm', 'Missing Alarm', 'ROC AUC']
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, mode, split_seed = model_group[0], model_group[1], model_group[2]
        
        targets = predictions['target_label'].values
        preds = predictions['pred_label'].values
        probs = predictions['pred_prob'].values
        
        conf_matrix = confusion_matrix(targets, preds)
        TN = conf_matrix[0][0]
        FN = conf_matrix[1][0]
        TP = conf_matrix[1][1]
        FP = conf_matrix[0][1]
        
        accuracy = (TP+TN) / (TP+TN+FP+FN)
        precision = TP / (TP+FP)
        recall = TP / (TP+FN)
        f1_score = 2 * ((precision*recall) / (precision+recall))
        false_alarm = FP / (TN+FP)
        missing_alarm = FN / (TP+FN)
        roc_auc_value = roc_auc_score(targets, probs)
        
        metrics.append([model_name, mode, split_seed, accuracy, precision, recall, f1_score, false_alarm, missing_alarm, roc_auc_value])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, grouping, metric_name='Accuracy'):
    mean_metrics = metrics.groupby(['Model', 'Mode'])[['Accuracy', 'Precision', 'Recall', 'F1', 'False Alarm', 'Missing Alarm', 'ROC AUC']].aggregate(['mean', 'std'])
    
    return mean_metrics


default_fields_dict = {
    'Mode': lambda x: 'frame-diff' if x == 'frame-difference' else 'color'
}

def render_to_latex(metrics, rename_func=default_fields_dict, **latex_kwargs):
    m = metrics.copy()
    # renaming
    for col, lambda_fn in rename_func.items():
        m[col] = m[col].apply(lambda_fn)
    m = m.groupby(['Model', 'Mode'])[['Accuracy', 'F1', 'False Alarm', 'Missing Alarm', 'ROC AUC']].aggregate(lambda x: u"{:.2f}\u00B1{:.2f}".format(x.mean(), x.std()))
    ltex = m.style.to_latex(
        **latex_kwargs
    )
    return ltex

In [ ]:
# Retrieve n names of videos not correctly predicted for each run

def retrieve_wrong_predictions(data, grouping, n=3):
    columns = ['Model', 'Video Names']
    video_names = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_name, predictions in grouped:
        model_name = model_name[0] + "_" + model_name[1] + "_" + "run-{}".format(model_name[2])
        
        ids = predictions['video_id'].values
        targets = predictions['target_label'].values
        preds = predictions['pred_label'].values
        probs = predictions['pred_prob'].values
        
        wrong_video_indexes = [i for i, (t, p) in enumerate(zip(targets, preds)) if t != p]
        
        n = n if n < len(wrong_video_indexes) else len(wrong_video_indexes)
        model_video_names = ""
        for i in range(n):
            model_video_names += "{}, ".format(ids[wrong_video_indexes[i]])
            
        video_names.append([model_name, model_video_names]) 
        
    wrong_videos_df = pd.DataFrame(video_names, columns=columns)
        
    return wrong_videos_df

<h1>Cross-Dataset Evaluation</h1>
<p>Train exploiting some datasets, test against the whole Bus Violence Dataset</p>

<h3>Pre-training on Surveillance Camera Fight</h3>

In [ ]:

ROOT = "/home/luca/dlearning/results/video-violence-detection/"

runs = {
    'BiConvLSTM-ECCV2018': list(Path(ROOT + '/runs/experiment=surveillance-camera-fight/').glob('biconvlstm-eccv*')),
    'ConvLSTM-AVSS2017': list(Path(ROOT + '/runs/experiment=surveillance-camera-fight').glob('convlstm-avss*')),
    'ResNet18-2+1D': list(Path(ROOT + '/runs/experiment=surveillance-camera-fight/').glob('resnet18-2plus1d*')),
    'ResNet18-3D': list(Path(ROOT + '/runs/experiment=surveillance-camera-fight/').glob('resnet18-3d*')),
    'SlowFast': list(Path(ROOT + '/runs/experiment=surveillance-camera-fight/').glob('slowfast*')),
    'Video-Swim-Transformer-k400': list(Path(ROOT + '/runs/experiment=surveillance-camera-fight/').glob('videoswintransformer-k400*')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_bus-violence.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'mode', 'split_seed']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics, model_grouper)

display(summary)

In [ ]:
# Obtain LaTeX table for the paper

latex = render_to_latex(
    metrics, 
    caption="Cross-Dataset evaluation (pre-training on Surveillance Camera Fight, test on Bus Dataset)",
    clines="skip-last;data",
    hrules=True,
    column_format="llccccc"
)

print(latex)

<h3>Pre-training on Real Life Violence Situations Dataset</h3>

In [ ]:
ROOT = "/home/luca/dlearning/results/video-violence-detection/"

runs = {
    'BiConvLSTM-ECCV2018': list(Path(ROOT + '/runs/experiment=real-life-violence/').glob('biconvlstm-*')),
    'ConvLSTM-AVSS2017': list(Path(ROOT + '/runs/experiment=real-life-violence/').glob('convlstm-*')),
    'ResNet18-2+1D': list(Path(ROOT + '/runs/experiment=real-life-violence/').glob('resnet18-2plus1d*')),
    'ResNet18-3D': list(Path(ROOT + '/runs/experiment=real-life-violence/').glob('resnet18-3d*')),
    'SlowFast': list(Path(ROOT + '/runs/experiment=real-life-violence/').glob('slowfast*')),
    'Video-Swim-Transformer-k400': list(Path(ROOT + '/runs/experiment=real-life-violence/').glob('videoswintransformer-k400*')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_bus-violence.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'mode', 'split_seed']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics, model_grouper)

display(summary)

In [ ]:
# Obtain LaTeX table for the paper

latex = render_to_latex(
    metrics, 
    caption="Cross-Dataset evaluation (pre-training on Real Life Violence Situations Dataset, test on Bus Dataset)",
    clines="skip-last;data",
    hrules=True,
    column_format="llccccc"
)

print(latex)

<h3>Pre-training on RWF-2000 Dataset</h3>

In [ ]:
ROOT = "/home/luca/dlearning/results/video-violence-detection/"

runs = {
    'BiConvLSTM-ECCV2018': list(Path(ROOT + '/runs/experiment=RWF-2000/').glob('biconvlstm-*')),
    'ConvLSTM-AVSS2017': list(Path(ROOT + '/runs/experiment=RWF-2000/').glob('convlstm-*')),
    'ResNet18-2+1D': list(Path(ROOT + '/runs/experiment=RWF-2000/').glob('resnet18-2plus1d*')),
    'ResNet18-3D': list(Path(ROOT + '/runs/experiment=RWF-2000/').glob('resnet18-3d*')),
    'SlowFast': list(Path(ROOT + '/runs/experiment=RWF-2000/').glob('slowfast*')),
    'Video-Swim-Transformer-k400': list(Path(ROOT + '/runs/experiment=RWF-2000/').glob('videoswintransformer-k400*')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_bus-violence.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'mode', 'split_seed']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics, model_grouper)

display(summary)

In [ ]:
# Obtain LaTeX table for the paper

latex = render_to_latex(
    metrics, 
    caption="Cross-Dataset evaluation (pre-training on RWF-2000 Dataset, test on Bus Dataset)",
    clines="skip-last;data",
    hrules=True,
    column_format="llccccc"
)

print(latex)

<h1>Draw some ROC curves</h1>

In [ ]:
# Draw a ROC curve for the given method

def compute_auc(target, prob):
    fpr, tpr, thresholds = metrics.roc_curve(target, prob)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc, tpr, fpr

def draw_roc_curve(data, grouping, model_name, colors, export_path='roc_plots', show_title=False, figsize=(4, 3.5)):
    fig, axs = plt.subplots(1, figsize=figsize)
    data = data.copy().reset_index()
    data = data.replace({'frame-difference': 'FD', 'rgb': 'RGB'})
    data = data[data['model'] == model_name]
    grouped = data.groupby(grouping)
    
    for model_group, all_predictions in grouped:
        dataset_name, mode = model_group[0], model_group[1]

        # find the run with best auc
        # aucs = all_predictions.groupby('split_seed').apply(lambda x: compute_auc(x['target_label'].values, x['pred_prob'].values)[0])
        # aucs = aucs.reset_index()
        # idxmax = aucs[0].idxmax()
        # chosen_seed = aucs.at[idxmax, 'split_seed']
        valid_preds = all_predictions #all_predictions[all_predictions['split_seed'] == chosen_seed]

        targets = valid_preds['target_label'].values
        probs = valid_preds['pred_prob'].values
        roc_auc, tpr, fpr = compute_auc(targets, probs)

        line_name = '{}-{}'.format(dataset_name, mode)
        linestyle= 'dashed' if mode == 'FD' else 'solid'
        display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=line_name)
        display.plot(ax=axs, linestyle=linestyle, color=colors[dataset_name])
        
    handles, labels = axs.get_legend_handles_labels()
    labels = [l[:-13] for l in labels]  # a bit hacky, remove auc from the legend
    axs.legend(handles, labels)
    axs.grid()
    if show_title:
        axs.set_title('ROC for {}'.format(model_name))
    if export_path is not None:
        export_path = Path(export_path)
        export_path.mkdir(parents=True, exist_ok=True)
        fig.tight_layout()
        fig.savefig(export_path / '{}.pdf'.format(model_name))
    
    

def collect_all_datasets(ROOT, model):
    model_folders = {
        'ResNet18-2+1D': 'resnet18-2plus1d',
        'ResNet18-3D': 'resnet18-3d',
        'ResNet18-MixedConvolution-3D': 'resnet18-mc-3d',
        'BiConvLSTM-ECCV2018': 'biconvlstm',
        'SlowFast': 'slowfast',
        'Video-Swim-Transformer-k400': 'videoswintransformer-k400',
        'ViViT-k400': 'vivit-k400',
        'DeVTR': 'devtr',
        'ConvLSTM-AVSS2017': 'convlstm',
    }

    model_folder = model_folders[model]
    pattern = '{}*'.format(model_folder)
    runs = {
        'SCF': list(Path(ROOT + '/runs/experiment=surveillance-camera-fight/').glob(pattern)),
        'RLV': list(Path(ROOT + '/runs/experiment=real-life-violence/').glob(pattern)),
        'RWF-2000': list(Path(ROOT + '/runs/experiment=RWF-2000/').glob(pattern))
    }

    dfs = []
    for k, v in runs.items():
        for r in v:
            df = collect_all(model, r, 'preds_bus-violence.csv')
            df['dataset'] = k
            dfs.append(df)

    predictions = pd.concat(dfs, ignore_index=True)
    return predictions

In [ ]:
ROOT = "/media/dlearning/shared/ciampi.messina/results/video-violence-detection/"

models = ['SlowFast', 'Video-Swim-Transformer-k400', 'ResNet18-3D', 'BiConvLSTM-ECCV2018']
colors = {"SCF": "gray", "RLV": "green", "RWF-2000": "blue"}
grouping = ['dataset', 'mode']

for model in models:
    predictions = collect_all_datasets(ROOT, model)
    draw_roc_curve(predictions, grouping, model, colors)